# Case Study 1 (Working file)

## Imports

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt

## Create DataFrame

In [2]:
column_names = ['time', 'scanMac', 'posX', 'posY', 'posZ', 'orientation', 'mac', 'signal', 'channel', 'type']

def parseDataFile(path):
    lines = None
    with open(path, 'r') as file:
        lines = file.readlines()
        file.close()

    instances = [] # will hold final data for dataframe

    for line in lines:

        # strip "\n" from line
        line = line.rstrip('\n')

        # skip if comment
        if line[0] == '#':
            continue

        base = [] # [t, id, x, y, z, degree]
        rows = [] # base + [mac, signal, channel, type]

        for keyvalue in line.split(';'):
            key, value = keyvalue.split('=')

            if key in ['t', 'id', 'degree']:
                base.append(value)
            elif key == 'pos':
                # pos (x, y, z)
                base += value.split(',')
            else:
                # mac addresses and metrics (signal, channel, type)
                row = base.copy()
                row.append(key)
                row += value.split(',')
                rows.append(row)

        instances += rows

    return pd.DataFrame(instances, columns = column_names)

df_offline = parseDataFile('../Data/offline.final.trace.txt')
df_online = parseDataFile('../Data/online.final.trace.txt')

### Offline Data

In [3]:
print("OFFLINE DATA")
print(df_offline.info())
df_offline.head()

OFFLINE DATA
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1181628 entries, 0 to 1181627
Data columns (total 10 columns):
time           1181628 non-null object
scanMac        1181628 non-null object
posX           1181628 non-null object
posY           1181628 non-null object
posZ           1181628 non-null object
orientation    1181628 non-null object
mac            1181628 non-null object
signal         1181628 non-null object
channel        1181628 non-null object
type           1181628 non-null object
dtypes: object(10)
memory usage: 90.2+ MB
None


,time,scanMac,posX,posY,posZ,orientation,mac,signal,channel,type
0,1139643118358,00:02:2D:21:0F:33,0.0,0.0,0.0,0.0,00:14:bf:b1:97:8a,-38,2437000000,3
1,1139643118358,00:02:2D:21:0F:33,0.0,0.0,0.0,0.0,00:14:bf:b1:97:90,-56,2427000000,3
2,1139643118358,00:02:2D:21:0F:33,0.0,0.0,0.0,0.0,00:0f:a3:39:e1:c0,-53,2462000000,3
3,1139643118358,00:02:2D:21:0F:33,0.0,0.0,0.0,0.0,00:14:bf:b1:97:8d,-65,2442000000,3
4,1139643118358,00:02:2D:21:0F:33,0.0,0.0,0.0,0.0,00:14:bf:b1:97:81,-65,2422000000,3


### Online Data

In [4]:
print(df_online.info())
df_online.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53303 entries, 0 to 53302
Data columns (total 10 columns):
time           53303 non-null object
scanMac        53303 non-null object
posX           53303 non-null object
posY           53303 non-null object
posZ           53303 non-null object
orientation    53303 non-null object
mac            53303 non-null object
signal         53303 non-null object
channel        53303 non-null object
type           53303 non-null object
dtypes: object(10)
memory usage: 4.1+ MB
None


,time,scanMac,posX,posY,posZ,orientation,mac,signal,channel,type
0,1139692477303,00:02:2D:21:0F:33,0.0,0.05,0.0,130.5,00:14:bf:b1:97:8a,-43,2437000000,3
1,1139692477303,00:02:2D:21:0F:33,0.0,0.05,0.0,130.5,00:0f:a3:39:e1:c0,-52,2462000000,3
2,1139692477303,00:02:2D:21:0F:33,0.0,0.05,0.0,130.5,00:14:bf:3b:c7:c6,-62,2432000000,3
3,1139692477303,00:02:2D:21:0F:33,0.0,0.05,0.0,130.5,00:14:bf:b1:97:81,-58,2422000000,3
4,1139692477303,00:02:2D:21:0F:33,0.0,0.05,0.0,130.5,00:14:bf:b1:97:8d,-62,2442000000,3


## Exploratory Data Analysis (EDA)

In [5]:
df_offline['mac'].value_counts()

00:0f:a3:39:e1:c0    145862
00:0f:a3:39:dd:cd    145619
00:14:bf:b1:97:8a    132962
00:14:bf:3b:c7:c6    126529
00:14:bf:b1:97:90    122315
00:14:bf:b1:97:8d    121325
00:14:bf:b1:97:81    120339
02:00:42:55:31:00    103887
02:64:fb:68:52:e6     50852
00:0f:a3:39:e0:4b     43508
02:2e:58:22:f1:ac     25112
00:0f:a3:39:e2:10     19162
02:37:fd:3b:54:b5      8732
02:b7:00:bb:a9:35      7602
02:5c:e0:50:49:de      6997
00:04:0e:5c:23:fc       418
00:30:bd:f8:7f:c5       301
00:e0:63:82:8b:a9       103
02:0a:3d:06:94:88         1
02:4f:99:43:30:cd         1
02:42:1c:4e:b5:c0         1
Name: mac, dtype: int64

## Long to Wide (Offline)

In [6]:
temp1 = df_offline.pivot(columns='mac', values='signal').fillna('')
temp2 = df_offline[['time', 'type']].join(temp1)
temp3 = temp2.groupby(['time', 'type']).agg(''.join)
df_offline_wide = df_offline[['time', 'posX', 'posY', 'posZ', 'orientation', 'type']].drop_duplicates().join(temp3, on=['time', 'type'])
df_offline_wide

,time,posX,posY,posZ,orientation,type,00:04:0e:5c:23:fc,00:0f:a3:39:dd:cd,00:0f:a3:39:e0:4b,00:0f:a3:39:e1:c0,...,00:e0:63:82:8b:a9,02:00:42:55:31:00,02:0a:3d:06:94:88,02:2e:58:22:f1:ac,02:37:fd:3b:54:b5,02:42:1c:4e:b5:c0,02:4f:99:43:30:cd,02:5c:e0:50:49:de,02:64:fb:68:52:e6,02:b7:00:bb:a9:35
0,1139643118358,0.0,0.0,0.0,0.0,3,,-75,-78,-53,...,,,,,,,,,,
9,1139643118358,0.0,0.0,0.0,0.0,1,,,,,...,,-84,,,,,,,-88,
11,1139643118744,0.0,0.0,0.0,0.0,3,,-73,-79,-54,...,,,,,,,,,,
20,1139643118744,0.0,0.0,0.0,0.0,1,,,,,...,,-85,,,,,,,,
21,1139643119002,0.0,0.0,0.0,0.0,3,,-65,-78,-54,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181612,1141936869952,20.0,8.0,0.0,315.7,1,,,,,...,,,,,,,,,,-73
1181613,1141936870204,20.0,8.0,0.0,315.7,3,,-77,,-53,...,,,,,,,,,,
1181619,1141936870204,20.0,8.0,0.0,315.7,1,,,,,...,,,,,,,,,,-73
1181620,1141936870456,20.0,8.0,0.0,315.7,3,,-76,,-53,...,,,,,,,,,,
